In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# example of horizontal shift image augmentation
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

train_datagen = ImageDataGenerator(
featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split = 0.2,
    dtype=None)

valid_datagen = ImageDataGenerator(
featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split = 0.2,
    dtype=None)

# prepare iterator
train = train_datagen.flow_from_directory(
    directory="../input/data-generation/data",
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",
    shuffle=True,
    subset='training')

valid = valid_datagen.flow_from_directory(
    directory="../input/data-generation/data",
    target_size=(200, 200),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",
    shuffle=True,
    subset='validation')


In [ ]:
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

In [ ]:
class ResNet:
  @staticmethod
  def residual_module(data, K, stride, chanDim, red=False,reg=0.0001, bnEps=2e-5, bnMom=0.9):
    shortcut = data
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,kernel_regularizer=l2(reg))(act1)
   
    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False,kernel_regularizer=l2(reg))(act2)
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

    if red:
      shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False,kernel_regularizer=l2(reg))(act1)
    x = add([conv3, shortcut])
    return x

  @staticmethod
  def build(width, height, depth, classes, stages, filters,reg=0.0001, bnEps=2e-5, bnMom=0.9):  
    inputShape = (height, width, depth)
    chanDim = -1
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
      chanDim = 1 

    inputs = Input(shape=inputShape)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(inputs)
    x = Conv2D(filters[0], (5, 5), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = ZeroPadding2D((1, 1))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    for i in range(0, len(stages)):
      stride = (1, 1) if i == 0 else (2, 2)
      x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)
      for j in range(0, stages[i] - 1):
        x = ResNet.residual_module(x, filters[i + 1],(1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

    x = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Activation("softmax")(x)
    
    model = Model(inputs,x, name="resnet")
    
    return model

In [ ]:
resnet = ResNet.build(200,200,3,2,[3,4,6,6],[64,128,256,512,512])

In [ ]:
resnet.summary()

In [ ]:
import tensorflow as tf
import keras
def scheduler(epoch, lr):
    if epoch<10:
        return lr
    else:
        return lr*0.996

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
resnet.compile(
        optimizer=tf.keras.optimizers.SGD(), loss="categorical_crossentropy", metrics=["accuracy"],run_eagerly=True
    )


#print(tf.keras.utils.plot_model(model))
hist=resnet.fit(train, validation_data = valid, epochs = 50, batch_size = 64,callbacks=[callback,tf.keras.CSVLogger('epochs.csv')])



In [ ]:
resnet.save("model-ishan-last1.h5")